<a href="https://colab.research.google.com/github/vrathod07/AI/blob/main/BettyAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pgeocode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for scraper: filename=scraper-0.1.0-py3-none-any.whl size=3486 sha256=4e5100c0820fb0afe3b9500e36e1f0009b2b6e60eb4b1dc1023e8d26e4b7cc22
  Stored in directory: /root/.cache/pip/wheels/da/8b/92/da1812cedf0e0d0cb672b1e603021ca30c0a104d34fc1b2a3a
Successfully built scraper


In [3]:
!pip install uszipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 7.8 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 977 kB/s 


In [4]:
import pandas as pd
import sys
import pgeocode as pgeo
from scraper import *
import numpy as np
import requests

from geopy.geocoders import Nominatim

In [5]:
df = pd.read_excel("BettyAI.xlsx")

In [6]:
df.head()

,Parcel_County,Parcel_Delinquent_Year,PCL_Code,PCL_Code_Year,Parcel_Number,PCL_Type,PCL_Status,PCL_Value,PCL_Date_Certified,PCL_Land_Sale_Date,...,State,ZipCode,Taxes,IDTaxes,Interest,Penalty,SC,CountyCost,RecordingFee,MunicipalLien
0,BENT,2016,10-1,2016,02-15415-000,LB,DL,2400,2019-05-10 13:40:29.077,2022-05-03,...,AR,72764-6415,652.80,0.0,170.20,65.28,136.95,4.0,15,0
1,BENT,2016,109-6,2016,22-00023-000,AC,DL,10,2019-05-10 13:40:29.077,2022-05-03,...,OK,74338,2.90,0.0,0.76,0.29,136.95,4.0,15,0
2,BENT,2016,29-5,2016,13-00184-517,AC,DL,1120,2019-05-10 13:40:29.077,2022-05-03,...,TX,78669,331.76,0.0,85.86,33.18,143.90,4.0,20,0
3,BENT,2016,33-5,2016,15-05253-000,LB,DL,780,2019-05-10 13:40:29.077,2022-05-03,...,KY,41031,189.65,0.0,47.82,18.97,143.90,4.0,15,0
4,BENT,2016,33-8,2016,15-06014-000,LB,DL,130,2019-05-10 13:40:29.077,2022-05-03,...,KY,41031,31.61,0.0,7.97,3.16,143.90,4.0,15,0


In [7]:
df.shape

(1914, 25)

In [8]:
df["Owner"]

0                                  DONERIGHT BUILDERS INC
1       MARY M EASLEY FAMILY TRUST%RUSSELL R & MARY M ...
2                                       SIDNEY M ROBINSON
3                                             JOYCE EUREN
4                                             JOYCE EUREN
                              ...                        
1909                TRACY JEANNETTE HUDSON%TRACY J HUDSON
1910                     JOHN COLLIN WATSON%JOHN C WATSON
1911                                MORRISON W Q PROP INC
1912    ALECIA DAVENPORT%CHRISTOPHER C & ALECIA DAVENPORT
1913    SUONG THACH & NGOC PHUONG BANH%SUONG & BANH THACH
Name: Owner, Length: 1914, dtype: object

In [9]:
df.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1914 entries, 0 to 1913
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Parcel_County           1914 non-null   object        
 1   Parcel_Delinquent_Year  1914 non-null   int64         
 2   PCL_Code                1914 non-null   object        
 3   PCL_Code_Year           1914 non-null   int64         
 4   Parcel_Number           1914 non-null   object        
 5   PCL_Type                1914 non-null   object        
 6   PCL_Status              1914 non-null   object        
 7   PCL_Value               1914 non-null   int64         
 8   PCL_Date_Certified      1914 non-null   datetime64[ns]
 9   PCL_Land_Sale_Date      1415 non-null   datetime64[ns]
 10  Legal                   1914 non-null   object        
 11  Owner                   1914 non-null   object        
 12  Address1                1914 non-null   object  

In [10]:
df.isnull().sum()

Parcel_County                0
Parcel_Delinquent_Year       0
PCL_Code                     0
PCL_Code_Year                0
Parcel_Number                0
PCL_Type                     0
PCL_Status                   0
PCL_Value                    0
PCL_Date_Certified           0
PCL_Land_Sale_Date         499
Legal                        0
Owner                        0
Address1                     0
Address2                  1899
City                         1
State                       11
ZipCode                     17
Taxes                        0
IDTaxes                      0
Interest                     0
Penalty                      0
SC                           0
CountyCost                   0
RecordingFee                 0
MunicipalLien                0
dtype: int64

In [11]:
df['Owner']

0                                  DONERIGHT BUILDERS INC
1       MARY M EASLEY FAMILY TRUST%RUSSELL R & MARY M ...
2                                       SIDNEY M ROBINSON
3                                             JOYCE EUREN
4                                             JOYCE EUREN
                              ...                        
1909                TRACY JEANNETTE HUDSON%TRACY J HUDSON
1910                     JOHN COLLIN WATSON%JOHN C WATSON
1911                                MORRISON W Q PROP INC
1912    ALECIA DAVENPORT%CHRISTOPHER C & ALECIA DAVENPORT
1913    SUONG THACH & NGOC PHUONG BANH%SUONG & BANH THACH
Name: Owner, Length: 1914, dtype: object

In [12]:
df.columns

Index(['Parcel_County', 'Parcel_Delinquent_Year', 'PCL_Code', 'PCL_Code_Year',
       'Parcel_Number', 'PCL_Type', 'PCL_Status', 'PCL_Value',
       'PCL_Date_Certified', 'PCL_Land_Sale_Date', 'Legal', 'Owner',
       'Address1', 'Address2', 'City', 'State', 'ZipCode', 'Taxes', 'IDTaxes',
       'Interest', 'Penalty', 'SC', 'CountyCost', 'RecordingFee',
       'MunicipalLien'],
      dtype='object')

In [13]:
df['City'].value_counts()

HOT SPRINGS    130
ROGERS          83
CONWAY          51
BENTONVILLE     50
GRAVETTE        46
              ... 
VISALIA          1
HINDSVILLE       1
AMARILLO         1
CENTENNIAL       1
Ward             1
Name: City, Length: 672, dtype: int64

In [14]:
df['State'].value_counts()

AR           851
FL           186
TX           171
CA            87
TN            58
OK            58
MO            54
IL            41
NY            32
LA            32
KS            27
UT            25
OH            20
NC            19
AZ            17
WI            17
IN            15
GA            14
NV            13
ID            10
IA            10
MA            10
MN            10
MS             9
CO             9
PA             9
WA             9
NJ             7
AL             7
MD             7
MI             7
KY             5
NE             5
VA             5
OR             5
HI             5
WY             4
INDONESIA      4
SC             4
NM             3
.              3
NH             2
               2
AK             2
PR             2
MT             2
SD             2
RI             2
ME             1
DC             1
..             1
CT             1
CANADA         1
Name: State, dtype: int64

In [15]:
for i in df['State']:
  if(type(i) == float):
    print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [16]:
df["City"] = df["City"].fillna(value="HOT SPRINGS")
df["State"] = df["State"].fillna(value="AR")

In [17]:
df["Parcel_County"].value_counts()

BENT    909
GARL    830
LONO     89
FAUL     86
Name: Parcel_County, dtype: int64

In [18]:
from uszipcode import SearchEngine
engine = SearchEngine()
for City, State in zip(df['City'],df['State']):
  try:
    zipcodes = engine.by_city_and_state(city=City, state=State)
    for zipcode in zipcodes:
        print(zipcode.zipcode, zipcode.major_city, zipcode.population)
        df['mail-zipcode'] = zipcode.zipcode
  except:
    zipcode = engine.by_state(state = State)
    for zipcode in zipcodes:
      df['mail-zipcode'] = zipcode.zipcode

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Download /root/.uszipcode/simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!
72762 Springdale 35342
72764 Springdale 50230
74338 Colcord 5480
78669 Spicewood 8731
41031 Cynthiana 15886
41031 Cynthiana 15886
73628 Cheyenne 1583
72015 Benton 27307
72019 Benton 23346
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
72601 Harrison 30163
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
48423 Davison 32708
48423 Davison 32708
75076 Pottsboro 737

72762 Springdale 35342
72764 Springdale 50230
72712 Bentonville 40933
72713 Bentonville None
72712 Bentonville 40933
72713 Bentonville None
71701 Camden 20286
37862 Sevierville 20934
37876 Sevierville 29246
92878 Corona None
92879 Corona 46745
92880 Corona 58763
92881 Corona 30991
92882 Corona 67917
33900 Fort Myers None
33901 Fort Myers 19500
33905 Fort Myers 31203
33907 Fort Myers 22138
33908 Fort Myers 38251
33900 Fort Myers None
33901 Fort Myers 19500
33905 Fort Myers 31203
33907 Fort Myers 22138
33908 Fort Myers 38251
33900 Fort Myers None
33901 Fort Myers 19500
33905 Fort Myers 31203
33907 Fort Myers 22138
33908 Fort Myers 38251
37862 Sevierville 20934
37876 Sevierville 29246
65020 Camdenton 14286
11020 Great Neck 5914
11021 Great Neck 17729
11023 Great Neck 9027
11024 Great Neck 8002
64014 Blue Springs 24797
64015 Blue Springs 31217
72762 Springdale 35342
72764 Springdale 50230
37862 Sevierville 20934
37876 Sevierville 29246
19040 Hatboro 20536
72762 Springdale 35342
72764 Sprin

75633 Carthage 13977
76681 Richland 873
33900 Fort Myers None
33901 Fort Myers 19500
33905 Fort Myers 31203
33907 Fort Myers 22138
33908 Fort Myers 38251
75442 Farmersville 8858
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
11020 Great Neck 5914
11021 Great Neck 17729
11023 Great Neck 9027
11024 Great Neck 8002
72712 Bentonville 40933
72713 Bentonville None
95037 Morgan Hill 45579
72730 Farmington 7585
72143 Searcy 35169
50251 Sully 1270


11020 Great Neck 5914
11021 Great Neck 17729
11023 Great Neck 9027
11024 Great Neck 8002
33122 Miami None
33125 Miami 52677
33126 Miami 46867
33127 Miami 28909
33128 Miami 7562
72901 Fort Smith 21591
72903 Fort Smith 26148
72904 Fort Smith 21893
72908 Fort Smith 13378
72916 Fort Smith 7131
72736 Gravette 6208
75633 Carthage 13977
75633 Carthage 13977
72736 Gravette 6208


72736 Gravette 6208
01247 North Adams 16087
72901 Fort Smith 21591
72903 Fort Smith 26148
72904 Fort Smith 21893
72908 Fort Smith 13378
72916 Fort Smith 7131
72901 Fort Smith 21591
72903 Fort Smith 26148
72904 Fort Smith 21893
72908 Fort Smith 13378
72916 Fort Smith 7131
72712 Bentonville 40933
72713 Bentonville None
66603 Topeka 2001
66604 Topeka 23344
66605 Topeka 19919
66606 Topeka 11284
66607 Topeka 10498
47901 Lafayette 3518
47904 Lafayette 16277
47905 Lafayette 38910
47909 Lafayette 39373
33701 Saint Petersburg 15014
33702 Saint Petersburg 29664
33703 Saint Petersburg 23680
33704 Saint Petersburg 16098
33705 Saint Petersburg 26435
85003 Phoenix 9369
85004 Phoenix 4965
85006 Phoenix 25742
85007 Phoenix 14040
85008 Phoenix 56145
60107 Streamwood 39927
72701 Fayetteville 39782
72703 Fayetteville 28261
72704 Fayetteville 21336
72756 Rogers 38455
72758 Rogers 33487
60502 Aurora 21873
60503 Aurora 16717
60504 Aurora 37919
60505 Aurora 76573
60506 Aurora 53013
66603 Topeka 2001
66604 To

32099 Jacksonville None
32202 Jacksonville 7915
32204 Jacksonville 6906
32205 Jacksonville 28225
32206 Jacksonville 17669
33019 Hollywood 15107
33020 Hollywood 41329
33021 Hollywood 45921
33023 Hollywood 63576
33024 Hollywood 63916
46140 Greenfield 38543
71949 Jessieville 1989
71901 Hot Springs National Park 29491
71913 Hot Springs National Park 43436
71951 Hot Springs National Park None
53066 Oconomowoc 33830
92501 Riverside 20970
92503 Riverside 84519
92504 Riverside 53778
92505 Riverside 47672
92506 Riverside 44001
33019 Hollywood 15107
33020 Hollywood 41329
33021 Hollywood 45921
33023 Hollywood 63576
33024 Hollywood 63916
27395 Greensboro None
27401 Greensboro 21932
27403 Greensboro 21958
27405 Greensboro 44567
27406 Greensboro 56458
75418 Bonham 14411
23111 Mechanicsville 36490
23116 Mechanicsville 27228
23111 Mechanicsville 36490
23116 Mechanicsville 27228
32137 Palm Coast 37821
32164 Palm Coast 41616
11520 Freeport 43341
71366 Saint Joseph 1974
32099 Jacksonville None
32202 Jack

In [19]:
x = {'BENT': 88314 ,'GARL':73703,'LONO':72104,'FAUL': 72034}

In [20]:
y = []
for i in df["Parcel_County"]:
  y.append(x[i])

df['pcl_zip'] = y

In [21]:
def get_age(row):
  if row['isBusiness']:
    return 0
  first_name = row['Owner'].split(' ')[0]
  try:
    response = requests.get(f"https://api.agify.io?name={first_name}&country_id=US&apikey=01eb88b54c29db19764c2d177d7b25c1", timeout=3)
  except Exception as e:
    print(e)
    pass
  try:
    age = response.json()['age']
  except:
    age = 0
  return age

def get_is_business(row):
    business_names = ['LLC', 'INC', 'TRUST', 'TRUSTEES', 'FAMILY']
    names = (str(row['Owner']).upper()).split(' ')
    print(names)
    return True if (np.in1d(business_names, names)).any() else False

def get_dist_btw_site_and_mail_zip(row, dist):
  try:
    mail_zip = int(row['mail-zipcode'])
    print("mail zip is: ", mail_zip)
    prop_zip = int(row['pcl_zip'])
    print("prop zip is: ", prop_zip)
  except Exception as e:
    print("one of the zip codes missing, in row ", e)
    return None

  print(mail_zip)
  print(prop_zip)
  return dist.query_postal_code(mail_zip, prop_zip)


In [22]:
dist = pgeo.GeoDistance('US')
df['potential_age'] = 0

df['isBusiness'] = df.apply(lambda row: get_is_business(row), axis=1)
df['dist_btw_site_mail_zip'] = df.apply(lambda row: get_dist_btw_site_and_mail_zip(row, dist), axis = 1)
df['potential_age'] = df.apply(lambda row: get_age(row), axis = 1)

print(df.shape)
#df.to_csv(file_path.split('.csv')[0]+'_with_more_cols.csv')


Streaming output truncated to the last 5000 lines.
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is:  88314
72086
88314
mail zip is:  72086
prop zip is: 

In [23]:
df.shape

(1914, 30)

In [24]:
df.head()

,Parcel_County,Parcel_Delinquent_Year,PCL_Code,PCL_Code_Year,Parcel_Number,PCL_Type,PCL_Status,PCL_Value,PCL_Date_Certified,PCL_Land_Sale_Date,...,Penalty,SC,CountyCost,RecordingFee,MunicipalLien,mail-zipcode,pcl_zip,potential_age,isBusiness,dist_btw_site_mail_zip
0,BENT,2016,10-1,2016,02-15415-000,LB,DL,2400,2019-05-10 13:40:29.077,2022-05-03,...,65.28,136.95,4.0,15,0,72086,88314,0.0,True,1297.477759
1,BENT,2016,109-6,2016,22-00023-000,AC,DL,10,2019-05-10 13:40:29.077,2022-05-03,...,0.29,136.95,4.0,15,0,72086,88314,0.0,True,1297.477759
2,BENT,2016,29-5,2016,13-00184-517,AC,DL,1120,2019-05-10 13:40:29.077,2022-05-03,...,33.18,143.90,4.0,20,0,72086,88314,64.0,False,1297.477759
3,BENT,2016,33-5,2016,15-05253-000,LB,DL,780,2019-05-10 13:40:29.077,2022-05-03,...,18.97,143.90,4.0,15,0,72086,88314,69.0,False,1297.477759
4,BENT,2016,33-8,2016,15-06014-000,LB,DL,130,2019-05-10 13:40:29.077,2022-05-03,...,3.16,143.90,4.0,15,0,72086,88314,69.0,False,1297.477759


In [ ]:
df.to_csv('new_file.csv')

In [ ]:
df.isnull().sum()

Parcel_County               0
Parcel_Delinquent_Year      0
PCL_Code                    0
PCL_Code_Year               0
Parcel_Number               0
PCL_Type                    0
PCL_Status                  0
PCL_Value                   0
PCL_Date_Certified          0
PCL_Land_Sale_Date        499
Legal                       0
Owner                       0
Address1                    0
City                        0
State                       0
ZipCode                    17
Taxes                       0
IDTaxes                     0
Interest                    0
Penalty                     0
SC                          0
CountyCost                  0
RecordingFee                0
MunicipalLien               0
Staet                       0
mail-zipcode                0
potential_age             129
isBusiness                  0
dist_btw_site_mail_zip      0
BETTY SCORE                 0
last_sale_deed_date       499
pcl_zip                     0
dtype: int64

In [ ]:
df_new = df

In [ ]:
df["Parcel_County"].value_counts()

BENT    909
GARL    830
LONO     89
FAUL     86
Name: Parcel_County, dtype: int64

In [25]:
df.columns

Index(['Parcel_County', 'Parcel_Delinquent_Year', 'PCL_Code', 'PCL_Code_Year',
       'Parcel_Number', 'PCL_Type', 'PCL_Status', 'PCL_Value',
       'PCL_Date_Certified', 'PCL_Land_Sale_Date', 'Legal', 'Owner',
       'Address1', 'Address2', 'City', 'State', 'ZipCode', 'Taxes', 'IDTaxes',
       'Interest', 'Penalty', 'SC', 'CountyCost', 'RecordingFee',
       'MunicipalLien', 'mail-zipcode', 'pcl_zip', 'potential_age',
       'isBusiness', 'dist_btw_site_mail_zip'],
      dtype='object')

In [26]:
df['BETTY SCORE'] = 100

df.loc[df['Owner'].isnull(), 'BETTY SCORE'] = df['BETTY SCORE'] - 50
df.loc[df['PCL_Land_Sale_Date'].isnull(), 'BETTY SCORE'] = df['BETTY SCORE'] - 50

# #df.loc[df['APN - UNFORMATTED'].isnull(), 'PROBABILITY_REASON'] = "NO_APN - UNFORMATTED"
# #df.loc[df['DO NOT MAIL'].str.strip().isin(['Y','X']) , 'BETTY SCORE'] = df['BETTY SCORE'] - 100
# #df.loc[df['DO NOT MAIL'].str.strip().isin(['Y','X']), 'PROBABILITY_REASON'] = "DO_NOT_MAIL"
# df.loc[df['FLOOD ZONE CODE'].str.strip().isin(['A', 'AE', 'AH', 'AO', 'V', 'VE']) , 'BETTY SCORE'] = df['BETTY SCORE'] - 10
# #df.loc[df['FLOOD ZONE CODE'].str.strip().isin(['A', 'AE', 'AH', 'AO', 'V', 'VE']) ,'PROBABILITY_REASON'] = "FLOOD_ZONE"
# df.loc[df['LOT AREA'] < 4500, 'BETTY SCORE'] = df['BETTY SCORE'] - 50


df['last_sale_deed_date'] = pd.to_datetime(df['PCL_Land_Sale_Date'])

df.loc[df['dist_btw_site_mail_zip'] < 30, 'BETTY SCORE'] = df['BETTY SCORE'] - 50
df.loc[df['Parcel_Delinquent_Year'] < 2018, 'BETTY SCORE'] = df['BETTY SCORE'] + 50
df.loc[df['dist_btw_site_mail_zip'] > 200, 'BETTY SCORE'] = df['BETTY SCORE'] + 60
df.loc[df['last_sale_deed_date'].dt.year > 2015, 'BETTY SCORE'] = df['BETTY SCORE'] - 60
df.loc[df['last_sale_deed_date'].dt.year < 2012, 'BETTY SCORE'] = df['BETTY SCORE'] + 60
df.loc[df['last_sale_deed_date'].isnull(), 'BETTY SCORE'] = df['BETTY SCORE'] + 60
#df.loc[df['OWNER 1 TYPE'] != 'Individual', 'BETTY SCORE'] = df['BETTY SCORE'] - 100

print(df.shape)
final_df = df.sort_values(by=['BETTY SCORE'], ascending=False)
final_df.drop(['last_sale_deed_date','dist_btw_site_mail_zip', 'IDTaxes'], axis=1, inplace = True)
final_df.to_csv('result.csv', index=False)


(1914, 32)


In [32]:
final_df.to_excel('result.xlsx', index=False)

In [28]:
final_df.head()

,Parcel_County,Parcel_Delinquent_Year,PCL_Code,PCL_Code_Year,Parcel_Number,PCL_Type,PCL_Status,PCL_Value,PCL_Date_Certified,PCL_Land_Sale_Date,...,Penalty,SC,CountyCost,RecordingFee,MunicipalLien,mail-zipcode,pcl_zip,potential_age,isBusiness,BETTY SCORE
995,GARL,2007,164-2,2007,71076,LB,DL,260,2010-10-28 09:25:50.010,2012-04-17,...,12.06,34.64,2.4,20,0,72086,73703,0.0,True,210
870,BENT,2019,6-2,2019,03-01746-000,LB,DL,15490,2022-05-10 09:54:46.460,NaT,...,103.44,30.00,4.0,15,0,72086,88314,51.0,False,170
603,BENT,2019,28-1,2019,16-16970-000,LB,DL,600,2022-05-10 09:54:46.460,NaT,...,7.64,30.00,4.0,15,0,72086,88314,69.0,False,170
592,BENT,2019,26-9,2019,16-15458-000,LB,DL,600,2022-05-10 09:54:46.460,NaT,...,7.64,30.00,4.0,15,0,72086,88314,NaN,False,170
593,BENT,2019,27-1,2019,16-15737-000,LB,DL,600,2022-05-10 09:54:46.460,NaT,...,6.28,30.00,4.0,15,0,72086,88314,50.0,False,170


In [31]:
df["BETTY SCORE"].max(),df["BETTY SCORE"].min()

(210, 40)